<a href="https://colab.research.google.com/github/nbht98/attack_cat_or_dog_classification/blob/master/TEST_CIFAR_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install mia

  Created wheel for mia: filename=mia-0.1.2-cp36-none-any.whl size=11079 sha256=055bb0997a4fe7fd557c870b694af3d92ef7648e024cac24be8a2a20ceab0b3e
  Stored in directory: /root/.cache/pip/wheels/e8/83/e4/baae7782aa0d2e45af485d25a7994bab3f76428e483252ce82
Successfully built mia


In [0]:
# Import library 
import numpy as np

from absl import app
from absl import flags

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

from mia.estimators import ShadowModelBundle, AttackModelBundle, prepare_attack_data


In [0]:
NUM_CLASSES = 10
WIDTH = 32
HEIGHT = 32
CHANNELS = 3

TARGET_EPOCHS = 100 # Number of epochs to train target and shadow models
ATTACK_EPOCHS = 100 # Number of epochs to train attack models.
NUM_SHADOWS = 3 # Number of epochs to train attack models.

In [0]:
def get_data():
    """Prepare CIFAR10 data."""
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)
    X_train = X_train.astype("float32")
    X_test = X_test.astype("float32")
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")
    X_train /= 255
    X_test /= 255
    return (X_train, y_train), (X_test, y_test)

In [0]:
def target_model_fn():
  """The architecture of the target (victim) model.
  The attack is white-box, hence the attacker is assumed to know this architecture too."""

  model = tf.keras.models.Sequential()

  model.add(
      layers.Conv2D(
          32,
          (3, 3),
          activation="relu",
          padding="same",
          input_shape=(WIDTH, HEIGHT, CHANNELS),
      )
  )
  model.add(layers.Conv2D(32, (3, 3), activation="relu"))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))

  model.add(layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
  model.add(layers.Conv2D(64, (3, 3), activation="relu"))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))

  model.add(layers.Flatten())

  model.add(layers.Dense(512, activation="relu"))
  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(NUM_CLASSES, activation="softmax"))
  model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])

  return model


def attack_model_fn():
  """Attack model that takes target model predictions and predicts membership.
  Following the original paper, this attack model is specific to the class of the input.
  AttachModelBundle creates multiple instances of this model for each class.
  """
  model = tf.keras.models.Sequential()

  model.add(layers.Dense(128, activation="relu", input_shape=(NUM_CLASSES,)))

  model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
  model.add(layers.Dense(64, activation="relu"))
  model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
  model.add(layers.Dense(64, activation="relu"))

  model.add(layers.Dense(1, activation="sigmoid"))
  model.compile("adam", loss="binary_crossentropy", metrics=["accuracy"])
  return model

In [5]:
(X_train, y_train), (X_test, y_test) = get_data()

170500096/170498071 [==============================] - 12s 0us/step


In [8]:
# Train the target model.
print("Training the target model...")
target_model = target_model_fn()
# target_model.fit(
#     X_train, y_train, epochs=TARGET_EPOCHS, validation_split=0.1, verbose=True
# )
target_model.load_weights("model900.h5")

Training the target model...


In [9]:
# We assume that attacker's data were not seen in target's training.
attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(
    X_test, y_test, test_size=0.1
)
print(attacker_X_train.shape, attacker_X_test.shape)

(9000, 32, 32, 3) (1000, 32, 32, 3)


In [0]:
SHADOW_DATASET_SIZE = 4000

# Train the shadow models.
smb = ShadowModelBundle(
    target_model_fn,
    shadow_dataset_size=SHADOW_DATASET_SIZE,
    num_models=NUM_SHADOWS,
)

In [14]:
print("Training the shadow models...")
X_shadow, y_shadow = smb.fit_transform(
    attacker_X_train,
    attacker_y_train,
    fit_kwargs=dict(
        epochs=TARGET_EPOCHS,
        verbose=True,
        validation_data=(attacker_X_test, attacker_y_test),
    ),
)

Training the shadow models...
Epoch 1/100
125/125 [==============================] - 1s 7ms/step - loss: 2.1191 - accuracy: 0.2030 - val_loss: 1.9058 - val_accuracy: 0.2980
Epoch 2/100
125/125 [==============================] - 1s 6ms/step - loss: 1.8572 - accuracy: 0.3063 - val_loss: 1.7612 - val_accuracy: 0.3400
Epoch 3/100
125/125 [==============================] - 1s 6ms/step - loss: 1.6635 - accuracy: 0.3810 - val_loss: 1.5812 - val_accuracy: 0.4340
Epoch 4/100
125/125 [==============================] - 1s 6ms/step - loss: 1.5255 - accuracy: 0.4423 - val_loss: 1.5404 - val_accuracy: 0.4360
Epoch 5/100
125/125 [==============================] - 1s 6ms/step - loss: 1.4738 - accuracy: 0.4518 - val_loss: 1.4803 - val_accuracy: 0.4610
Epoch 6/100
125/125 [==============================] - 1s 6ms/step - loss: 1.3539 - accuracy: 0.5067 - val_loss: 1.4409 - val_accuracy: 0.4670
Epoch 7/100
125/125 [==============================] - 1s 6ms/step - loss: 1.2710 - accuracy: 0.5447 - val_loss:

In [0]:
# ShadowModelBundle returns data in the format suitable for the AttackModelBundle.
amb = AttackModelBundle(attack_model_fn, num_classes=NUM_CLASSES)

In [16]:
# Fit the attack models.
print("Training the attack models...")
amb.fit(
    X_shadow, y_shadow, fit_kwargs=dict(epochs=ATTACK_EPOCHS, verbose=True)
)

Training the attack models...
Epoch 1/100
74/74 [==============================] - 0s 3ms/step - loss: 0.6143 - accuracy: 0.6684
Epoch 2/100
74/74 [==============================] - 0s 3ms/step - loss: 0.4993 - accuracy: 0.7322
Epoch 3/100
74/74 [==============================] - 0s 3ms/step - loss: 0.4831 - accuracy: 0.7482
Epoch 4/100
74/74 [==============================] - 0s 3ms/step - loss: 0.4740 - accuracy: 0.7529
Epoch 5/100
74/74 [==============================] - 0s 3ms/step - loss: 0.4621 - accuracy: 0.7681
Epoch 6/100
74/74 [==============================] - 0s 3ms/step - loss: 0.4639 - accuracy: 0.7596
Epoch 7/100
74/74 [==============================] - 0s 3ms/step - loss: 0.4544 - accuracy: 0.7757
Epoch 8/100
74/74 [==============================] - 0s 3ms/step - loss: 0.4452 - accuracy: 0.7812
Epoch 9/100
74/74 [==============================] - 0s 3ms/step - loss: 0.4475 - accuracy: 0.7812
Epoch 10/100
74/74 [==============================] - 0s 3ms/step - loss: 0.443

In [19]:
# Test the success of the attack.
ATTACK_TEST_DATASET_SIZE = 4000
# Prepare examples that were in the training, and out of the training.
data_in = X_train[:ATTACK_TEST_DATASET_SIZE], y_train[:ATTACK_TEST_DATASET_SIZE]
data_out = X_test[:ATTACK_TEST_DATASET_SIZE], y_test[:ATTACK_TEST_DATASET_SIZE]

# Compile them into the expected format for the AttackModelBundle.
attack_test_data, real_membership_labels = prepare_attack_data(
    target_model, data_in, data_out
)

# Compute the attack accuracy.
attack_guesses = amb.predict(attack_test_data)
attack_accuracy = np.mean(attack_guesses == real_membership_labels)

print(attack_accuracy)

0.66925
